<a href="https://colab.research.google.com/github/nskalyan-ops/iet_ev/blob/main/Ev_charge_stations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports

In [ ]:
!pip install torch torchvision torchaudio torch-geometric networkx


In [ ]:
import pandas as pd

# Load datasets
df_users = pd.read_csv("ev_users_training_data.csv")
df_stations = pd.read_csv("ev_stations.csv")

# Print sample data
df_users.head(), df_stations.head()


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx

# Convert user & station locations to tensor format
user_tensor = torch.tensor(df_users[["latitude", "longitude"]].values, dtype=torch.float)
station_tensor = torch.tensor(df_stations[["latitude", "longitude"]].values, dtype=torch.float)

# Build graph with NetworkX
G = nx.Graph()

# Add users & stations as nodes
for i in range(len(df_users)):
    G.add_node(f"user_{i}", x=user_tensor[i])

for i in range(len(df_stations)):
    G.add_node(f"station_{i}", x=station_tensor[i])

# Add edges (user → recommended station)
for i, row in df_users.iterrows():
    recommended_idx = df_stations.index[df_stations["station_id"] == row["recommended_station_id"]].tolist()[0]
    G.add_edge(f"user_{i}", f"station_{recommended_idx}")

# Convert to PyTorch Geometric graph
graph_data = from_networkx(G)

# Save graph data
torch.save(graph_data, "ev_graph_data.pt")

graph_data


In [ ]:
import torch.nn as nn
import torch_geometric.nn as pyg_nn

class EVRecommendationGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EVRecommendationGNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(input_dim, hidden_dim)
        self.conv2 = pyg_nn.GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Initialize model
model = EVRecommendationGNN(input_dim=2, hidden_dim=16, output_dim=len(df_stations))


In [ ]:
import torch.optim as optim

device = torch.device("cpu")
graph_data = graph_data.to(device)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(graph_data)

    # Loss calculation (dummy labels for now, replace with real labels)
    num_classes = len(df_stations)
    target = torch.randint(0, num_classes, (len(df_users),), dtype=torch.long).to(device)

    loss = criterion(output[:len(df_users)], target)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


In [ ]:
!pip install gym torch stable-baselines3


In [ ]:
import gym
import numpy as np
import torch

class EVChargingEnv(gym.Env):
    def __init__(self, df_users, df_stations):
        super(EVChargingEnv, self).__init__()

        # State: [user_lat, user_lon, battery_level]
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(3,), dtype=np.float32)

        # Actions: Recommend a charging station (0 to num_stations-1)
        self.action_space = gym.spaces.Discrete(len(df_stations))

        self.df_users = df_users
        self.df_stations = df_stations
        self.current_user_idx = 0  # Track which user is being served

    def reset(self):
        """Reset environment for a new episode"""
        self.current_user_idx = np.random.randint(0, len(self.df_users))
        user = self.df_users.iloc[self.current_user_idx]
        state = np.array([user["latitude"], user["longitude"], user["battery_level"]], dtype=np.float32)
        return state

    def step(self, action):
        """Take an action (recommend a station) and return the reward"""
        user = self.df_users.iloc[self.current_user_idx]
        station = self.df_stations.iloc[action]

        # Compute distance (negative reward for far stations)
        distance = np.linalg.norm([user["latitude"] - station["latitude"], user["longitude"] - station["longitude"]])

        # Check station availability
        available = station["vacancy"] > 0

        # Reward function
        reward = 0
        if available:
            reward += 10  # Positive reward for recommending an available station
        else:
            reward -= 5  # Negative reward for sending user to a full station

        # Negative reward for long distances
        reward -= distance * 5

        done = True  # One recommendation per episode
        return np.array([user["latitude"], user["longitude"], user["battery_level"]], dtype=np.float32), reward, done, {}





In [ ]:
# # Create an agent
# agent = EVChargingRLAgent(num_stations=len(df_stations))

# # Train the model
# agent.train(df_users, df_stations, num_epochs=200)


In [ ]:
from stable_baselines3 import DQN
!pip install shimmy>=2.0

# Initialize the environment
env = EVChargingEnv(df_users, df_stations)

# Train the RL model
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Save the trained model
model.save("ev_rl_model")


In [ ]:
# Load trained RL model
model = DQN.load("ev_rl_model")

# Test with a random user
obs = env.reset()
action, _ = model.predict(obs)
recommended_station = df_stations.iloc[action]

print("Recommended Station:", recommended_station["station_id"])


In [ ]:
!pip install matplotlib seaborn folium


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap


In [ ]:
# # Create a base map centered around user locations
# m = folium.Map(location=[df_users["latitude"].mean(), df_users["longitude"].mean()], zoom_start=12)

# # Add EV Users (Blue Markers)
# for _, user in df_users.iterrows():
#     folium.Marker(
#         location=[user["latitude"], user["longitude"]],
#         popup=f"User {user['user_id']} (Battery: {user['battery_level']*100:.1f}%)",
#         icon=folium.Icon(color="blue", icon="user")
#     ).add_to(m)

# # Add Charging Stations (Red Markers)
# for _, station in df_stations.iterrows():
#     folium.Marker(
#         location=[station["latitude"], station["longitude"]],
#         popup=f"Station {station['station_id']} (Vacancy: {station['vacancy']})",
#         icon=folium.Icon(color="red", icon="bolt")
#     ).add_to(m)

# # Show map
# m


In [ ]:
# Prepare data for heatmap
heatmap_data = [[row["latitude"], row["longitude"]] for _, row in df_users.iterrows()]

# Add heatmap layer
m_heatmap = folium.Map(location=[df_users["latitude"].mean(), df_users["longitude"].mean()], zoom_start=12)
HeatMap(heatmap_data, radius=15).add_to(m_heatmap)

# Show heatmap
m_heatmap


In [ ]:
def test_specific_user(user_id):
    """Test model recommendation for a specific user"""
    user = df_users[df_users["user_id"] == user_id].iloc[0]
    obs = np.array([user["latitude"], user["longitude"], user["battery_level"]], dtype=np.float32)

    # Predict charging station using RL model
    action, _ = model.predict(obs)
    recommended_station = df_stations.iloc[action]

    # Print result
    print(f"User {user_id} -> Recommended Station: {recommended_station['station_id']}")

    return user, recommended_station

# Example: Test for user ID 5
user_data, station_data = test_specific_user("314e7455-280d-42e3-8699-74ac773ce198")




In [ ]:
# Select the user
selected_user_id = "314e7455-280d-42e3-8699-74ac773ce198"
user_data = df_users[df_users["user_id"] == selected_user_id].iloc[0]

# Get model recommendation
obs = np.array([user_data["latitude"], user_data["longitude"], user_data["battery_level"]], dtype=np.float32)
action, _ = model.predict(obs)
recommended_station = df_stations.iloc[action]

# Print results
print(f"User ID: {selected_user_id}")
print(f"User Location: ({user_data['latitude']}, {user_data['longitude']})")
print(f"Battery Level: {user_data['battery_level']*100:.1f}%")
print(f"Recommended Station ID: {recommended_station['station_id']}")
print(f"Station Location: ({recommended_station['latitude']}, {recommended_station['longitude']})")
print(f"Vacancy: {recommended_station['vacancy']}")
print(f"Wait Time: {recommended_station['wait_time_min']} min")


In [ ]:
import numpy as np

def find_best_station(user_data, df_stations):
    """Find the optimal charging station based on distance, vacancy, and wait time."""
    user_location = np.array([user_data["latitude"], user_data["longitude"]])

    best_station = None
    best_score = float('inf')  # Lower is better

    for _, station in df_stations.iterrows():
        station_location = np.array([station["latitude"], station["longitude"]])
        distance = np.linalg.norm(user_location - station_location)  # Euclidean distance

        # Ignore stations with no vacancy if the user is not in an urgent state
        if station["vacancy"] == 0 and user_data["battery_level"] > 0.2:
            continue

        # Calculate a ranking score based on distance & wait time
        score = distance + (station["wait_time_min"] / 10)  # Normalize wait time

        if score < best_score:
            best_score = score
            best_station = station

    return best_station

# Find the best station based on our logic
optimal_station = find_best_station(user_data, df_stations)

print(f"Optimal Station ID: {optimal_station['station_id']}")
print(f"Optimal Station Distance: {np.linalg.norm([user_data['latitude'] - optimal_station['latitude'], user_data['longitude'] - optimal_station['longitude']])}")
print(f"Optimal Station Vacancy: {optimal_station['vacancy']}")
print(f"Optimal Station Wait Time: {optimal_station['wait_time_min']} min")


In [ ]:
# Map centered at user's location
m_verify = folium.Map(location=[user_data["latitude"], user_data["longitude"]], zoom_start=13)

# Add user location (Green)
folium.Marker(
    location=[user_data["latitude"], user_data["longitude"]],
    popup=f"User {selected_user_id} (Battery: {user_data['battery_level']*100:.1f}%)",
    icon=folium.Icon(color="green", icon="user")
).add_to(m_verify)

# Add recommended station (Purple)
folium.Marker(
    location=[recommended_station["latitude"], recommended_station["longitude"]],
    popup=f"Model Recommendation: Station {recommended_station['station_id']} (Vacancy: {recommended_station['vacancy']})",
    icon=folium.Icon(color="purple", icon="bolt")
).add_to(m_verify)

# Add optimal station (Blue)
folium.Marker(
    location=[optimal_station["latitude"], optimal_station["longitude"]],
    popup=f"Best Station: Station {optimal_station['station_id']} (Vacancy: {optimal_station['vacancy']})",
    icon=folium.Icon(color="blue", icon="bolt")
).add_to(m_verify)

# Show the verification map
m_verify
